This notebook is used to calculate the performance metrics based on raw predictions. <br>

In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
import scipy

In [2]:
import warnings
warnings.filterwarnings('ignore')

-  a function for metric calculation

In [3]:
def metric_calc(labels, preds, metric='RMSE'):
    # convert the labels and preds to list
    labels = list(labels)
    preds = list(preds)
    # calculate score
    if metric == 'RMSE':
        score = metrics.mean_squared_error(labels, preds, squared=False)
    elif metric == 'MAE':
        score = metrics.mean_absolute_error(labels, preds) 
    elif metric == 'R2':
        score = metrics.r2_score(labels, preds)
    elif metric == 'Pearson_R':
        score = scipy.stats.pearsonr(labels, preds)[0] 
    elif metric == 'AUROC':
        try:
            score = metrics.roc_auc_score(labels, preds) 
        except ValueError:
            score = -1
    elif metric == 'AUPRC':
        # auc based on precision_recall curve
        precision, recall, _ = metrics.precision_recall_curve(labels, preds)
        score = metrics.auc(recall, precision)
    elif metric == 'Precision_PPV':
        # get the roc curve points
        false_pos_rate, true_pos_rate, proba = metrics.roc_curve(labels, preds)
        # calculate the optimal probability cutoff using Youden's J statistic with equal weight to FP and FN
        optimal_proba_cutoff = sorted(list(zip(np.abs(true_pos_rate - false_pos_rate), proba)),\
                                      key=lambda i: i[0], reverse=True)[0][1]
        # get hard_preds
        hard_preds = [1 if p > optimal_proba_cutoff else 0 for p in preds]
        # calculate precision based on hard_preds - pos_label as 1
        score = metrics.precision_score(labels, hard_preds, pos_label=1)
    elif metric == 'Precision_NPV':
        # get the roc curve points
        false_pos_rate, true_pos_rate, proba = metrics.roc_curve(labels, preds)
        # calculate the optimal probability cutoff using Youden's J statistic with equal weight to FP and FN
        optimal_proba_cutoff = sorted(list(zip(np.abs(true_pos_rate - false_pos_rate), proba)),\
                                      key=lambda i: i[0], reverse=True)[0][1]
        # get hard_preds
        hard_preds = [1 if p > optimal_proba_cutoff else 0 for p in preds]
        # calculate precision based on hard_preds - pos_label as 0
        score = metrics.precision_score(labels, hard_preds, pos_label=0)
        
    return score

# Calculate performance metrics on all molecules

## Benchmark datasets

In [4]:
#specify the dataset details
folder = 'benchmark'
task_setting = 'benchmark'
mol_props = ['BACE', 'BBBP', 'HIV', 'ESOL', 'FreeSolv',  'Lipop'] 
split_types = ['scaffold', 'random']
num_folds = 30

In [5]:
#make an empty dataframe to attach the results
perf_df = pd.DataFrame(columns=['metric_score', 'metric_name', 'mol_prop', 'model_name','fold', 'split_type', 'task'])

In [6]:
for mol_prop in mol_props:
    #get the right set of metric_names
    if mol_prop in ['BACE', 'BBBP', 'HIV']:
        metric_names =  ['AUROC', 'AUPRC', 'Precision_PPV', 'Precision_NPV']
    elif mol_prop in ['ESOL', 'FreeSolv', 'Lipop']:
        metric_names = ['RMSE', 'MAE', 'R2', 'Pearson_R']
    for split_type in split_types:
        for fold in range(num_folds):
            for metric_name in metric_names:
                #colnames: SMILES	labels	preds
                preds_rf = pd.read_csv('../results/raw_predictions/RF/{task}/{mol_prop}/{split_type}/r2_b2048_test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #drop NaN values
                preds_rf.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_rf['labels'], preds_rf['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'RF', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

                #colnames: SMILES	labels	preds
                preds_molbert = pd.read_csv('../results/raw_predictions/molbert/{task}/{mol_prop}/molbert/{split_type}/test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #drop NaN values
                preds_molbert.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_molbert['labels'], preds_molbert['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'molbert', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

                #colnames: SMILES	labels	preds
                preds_grover = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base/{split_type}/test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #ensure the labels and preds back to double type
                preds_grover['labels'] = preds_grover['labels'].astype(float)
                preds_grover['preds'] = preds_grover['preds'].astype(float)
                #drop NaN values
                preds_grover.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover['labels'], preds_grover['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

                #colnames: SMILES	labels	preds
                preds_grover_rdkit = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base_rdkit/{split_type}/test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #ensure the labels and preds back to double type
                preds_grover_rdkit['labels'] = preds_grover_rdkit['labels'].astype(float)
                preds_grover_rdkit['preds'] = preds_grover_rdkit['preds'].astype(float)
                #drop NaN values
                preds_grover_rdkit.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover_rdkit['labels'], preds_grover_rdkit['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base_rdkit', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

In [7]:
perf_df.loc[(perf_df['mol_prop']=='HIV') & (perf_df['metric_name']=='Precision_PPV')].head()

,metric_score,metric_name,mol_prop,model_name,fold,split_type,task
1928,0.168803,Precision_PPV,HIV,RF,0,scaffold,benchmark
1929,0.081454,Precision_PPV,HIV,molbert,0,scaffold,benchmark
1930,0.036694,Precision_PPV,HIV,grover_base,0,scaffold,benchmark
1931,0.096070,Precision_PPV,HIV,grover_base_rdkit,0,scaffold,benchmark
1944,0.313291,Precision_PPV,HIV,RF,1,scaffold,benchmark


In [8]:
# save to results directory
perf_df.to_csv('../results/processed_performance/{folder}_grand_perf_df_{task}.csv'.format(folder=folder, task=task_setting), index=False)

## Opioids datasets

In [14]:
#specify the dataset details
folder = 'opioids'
task_setting = 'cutoff6' # reg, cutoff6
mol_props = ['MDR1', 'CYP2D6', 'CYP3A4', 'MOR', 'DOR', 'KOR'] 
split_types = ['scaffold', 'random']
num_folds = 30

-  prediction over all molecules

In [15]:
#make an empty dataframe to attach the results
perf_df = pd.DataFrame(columns=['metric_score', 'metric_name', 'mol_prop', 'model_name','fold', 'split_type', 'task'])

In [16]:
for mol_prop in mol_props:
    #get the right set of metric_names
    if task_setting == 'cutoff6':
        metric_names =  ['AUROC', 'AUPRC', 'Precision_PPV', 'Precision_NPV']
    elif task_setting == 'reg':
        metric_names = ['RMSE', 'MAE', 'R2', 'Pearson_R']
    for split_type in split_types:
        for fold in range(num_folds):
            for metric_name in metric_names:
                #colnames: SMILES	labels	preds
                preds_rf = pd.read_csv('../results/raw_predictions/RF/{task}/{mol_prop}/{split_type}/r2_b2048_test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #drop NaN values
                preds_rf.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_rf['labels'], preds_rf['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'RF', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

                #colnames: SMILES	labels	preds
                preds_molbert = pd.read_csv('../results/raw_predictions/molbert/{task}/{mol_prop}/molbert/{split_type}/test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #drop NaN values
                preds_molbert.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_molbert['labels'], preds_molbert['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'molbert', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

                #colnames: SMILES	labels	preds
                preds_grover = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base/{split_type}/test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #convert the labels and preds back to double type
                preds_grover['labels'] = preds_grover['labels'].astype(float)
                preds_grover['preds'] = preds_grover['preds'].astype(float)
                #drop NaN values
                preds_grover.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover['labels'], preds_grover['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

                #colnames: SMILES	labels	preds
                preds_grover_rdkit = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base_rdkit/{split_type}/test_result_fold{fold}.csv'\
                                          .format(fold=fold, mol_prop=mol_prop, split_type=split_type, task=task_setting))
                #convert the labels and preds back to double type
                preds_grover_rdkit['labels'] = preds_grover_rdkit['labels'].astype(float)
                preds_grover_rdkit['preds'] = preds_grover_rdkit['preds'].astype(float)
                #drop NaN values
                preds_grover_rdkit.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover_rdkit['labels'], preds_grover_rdkit['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base_rdkit', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                perf_df = perf_df.append(row_to_add, ignore_index=True)

In [17]:
perf_df.loc[(perf_df['mol_prop']=='MDR1') & (perf_df['metric_name']=='RMSE')].head()

,metric_score,metric_name,mol_prop,model_name,fold,split_type,task


In [18]:
# save to results directory
perf_df.to_csv('../results/processed_performance/{folder}_grand_perf_df_{task}.csv'.format(folder=folder, task=task_setting), index=False)

# Calculate performance metrics for AC and non-AC molecules

-  Note: AC molecules are those with scaffolds that have pIC50 spanning two magnitudes<br>


## Opioids datasets

In [25]:
# specify datasets details
folder = 'opioids'
mol_props = ['MDR1', 'CYP2D6', 'CYP3A4', 'MOR', 'DOR', 'KOR'] 
task_setting = 'reg' # reg, cutoff6
split_types = ['scaffold', 'random']
num_folds = 30

-  Note: use **Structure_Analysis.ipynb** to generate the scaffolds_freq.csv and structural_traits.csv for each molecular property

In [26]:
# get the AC scaffolds first for each dataset
AC_molecules_list = []

for mol_prop in mol_props:
    #read the scaffolds_freq df
    scaffold_freq = pd.read_csv('../results/structures/{folder}/{name}_scaffolds_freq.csv'.format(folder=folder, name=mol_prop))
    #get the Murcko-Bemis scaffolds in each dataset
    scaffolds = list(scaffold_freq['SMILES'])
    
    #read the structtraits df
    scaff_data = pd.read_csv('../results/structures/{folder}/{name}_structural_traits.csv'.format(folder=folder, name=mol_prop))
    #only keep the molecules in scaffolds for AC_scaffolds generation
    scaff_data = scaff_data[scaff_data['scaffold'].isin(scaffolds)]
    
    #make a var to store the AC scaffolds
    AC_scaffolds = []
    #iterate through all scaffolds and collect the AC_scaffolds
    for scaffold in scaffolds:
        min_pIC50 = min(scaff_data[scaff_data['scaffold']==scaffold]['label'])
        max_pIC50 = max(scaff_data[scaff_data['scaffold']==scaffold]['label'])
        
        #when the min-max spans at least two, it is an AC scaffold
        if max_pIC50 - min_pIC50 >= 2:
            AC_scaffolds.append(scaffold)
        
    #get the molecules with AC scaffolds
    AC_molecules = list(scaff_data[scaff_data['scaffold'].isin(AC_scaffolds)]['SMILES'])
            
    #append the AC_scaffolds to the AC_scaffolds_list
    AC_molecules_list.append(AC_molecules)

-  for AC molecules

In [27]:
#make an empty dataframe to attach the results for AC molecules
AC_perf_df = pd.DataFrame(columns=['metric_score', 'metric_name', 'mol_prop', 'model_name','fold', 'split_type', 'task'])

for mol_prop in mol_props:
    #get the corresponding AC molecules for each mol_prop
    AC_molecules = AC_molecules_list[mol_props.index(mol_prop)]
    #get the right set of metric_names
    if task_setting == 'cutoff6':
        metric_names =  ['AUROC', 'AUPRC', 'Precision_PPV', 'Precision_NPV']
    elif task_setting == 'reg':
        metric_names = ['RMSE', 'MAE', 'R2', 'Pearson_R']
    for split_type in split_types:
        for fold in range(num_folds):
            for metric_name in metric_names:
                ##read RF raw preds
                preds_rf = pd.read_csv('../results/raw_predictions/RF/{task}/{mol_prop}/{split_type}/r2_b2048_test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES in the AC set
                preds_rf = preds_rf[preds_rf['SMILES'].isin(AC_molecules)]
                #drop NaN values
                preds_rf.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_rf['labels'], preds_rf['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'RF', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                AC_perf_df = AC_perf_df.append(row_to_add, ignore_index=True)

                ##read molbert raw preds
                preds_molbert = pd.read_csv('../results/raw_predictions/molbert/{task}/{mol_prop}/molbert/{split_type}/test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES in the AC set
                preds_molbert = preds_molbert[preds_molbert['SMILES'].isin(AC_molecules)]
                #check if the column names of preds_molbert are 'label_targets', 'label_preds'
                if list(preds_molbert.columns) == ['label_targets', 'label_preds']:
                    #change column names
                    preds_molbert.columns = ['labels', 'preds']
                #drop NaN values
                preds_molbert.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_molbert['labels'], preds_molbert['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'molbert', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                AC_perf_df = AC_perf_df.append(row_to_add, ignore_index=True)
                
                ##read grover raw preds
                preds_grover = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base/{split_type}/test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES in the AC set
                preds_grover = preds_grover[preds_grover['SMILES'].isin(AC_molecules)]
                #convert the labels and preds back to double type
                preds_grover['labels'] = preds_grover['labels'].astype(float)
                preds_grover['preds'] = preds_grover['preds'].astype(float)
                #drop NaN values
                preds_grover.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover['labels'], preds_grover['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                AC_perf_df = AC_perf_df.append(row_to_add, ignore_index=True)

                ##read grover_rdkit raw preds
                preds_grover_rdkit = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base_rdkit/{split_type}/test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES in the AC set
                preds_grover_rdkit = preds_grover_rdkit[preds_grover_rdkit['SMILES'].isin(AC_molecules)]
                #convert the labels and preds back to double type
                preds_grover_rdkit['labels'] = preds_grover_rdkit['labels'].astype(float)
                preds_grover_rdkit['preds'] = preds_grover_rdkit['preds'].astype(float)
                #drop NaN values
                preds_grover_rdkit.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover_rdkit['labels'], preds_grover_rdkit['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base_rdkit', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                AC_perf_df = AC_perf_df.append(row_to_add, ignore_index=True)

-  for non-AC molecules

In [28]:
#make an empty dataframe to attach the results for non-AC molecules
nonAC_perf_df = pd.DataFrame(columns=['metric_score', 'metric_name', 'mol_prop', 'model_name','fold', 'split_type', 'suffix'])

for mol_prop in mol_props:
    #get the corresponding AC molecules for each mol_prop
    AC_molecules = AC_molecules_list[mol_props.index(mol_prop)]
    #get the right set of metric_names
    if task_setting == 'cutoff6':
        metric_names =  ['AUROC', 'AUPRC', 'Precision_PPV', 'Precision_NPV']
    elif task_setting == 'reg':
        metric_names = ['RMSE', 'MAE', 'R2', 'Pearson_R']
    for split_type in split_types:
        for fold in range(num_folds):
            for metric_name in metric_names:
                ##read RF raw preds
                preds_rf = pd.read_csv('../results/raw_predictions/RF/{task}/{mol_prop}/{split_type}/r2_b2048_test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES not in the AC set
                preds_rf = preds_rf[~preds_rf['SMILES'].isin(AC_molecules)]
                #drop NaN values
                preds_rf.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_rf['labels'], preds_rf['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'RF', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                nonAC_perf_df = nonAC_perf_df.append(row_to_add, ignore_index=True)

                ##read molbert raw preds
                preds_molbert = pd.read_csv('../results/raw_predictions/molbert/{task}/{mol_prop}/molbert/{split_type}/test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES not in the AC set
                preds_molbert = preds_molbert[~preds_molbert['SMILES'].isin(AC_molecules)]
                #check if the column names of preds_molbert are 'label_targets', 'label_preds'
                if list(preds_molbert.columns) == ['label_targets', 'label_preds']:
                    #change column names
                    preds_molbert.columns = ['labels', 'preds']
                #drop NaN values
                preds_molbert.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_molbert['labels'], preds_molbert['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'molbert', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                nonAC_perf_df = nonAC_perf_df.append(row_to_add, ignore_index=True)
                
                ##read grover raw preds
                preds_grover = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base/{split_type}/test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES not in the AC set
                preds_grover = preds_grover[~preds_grover['SMILES'].isin(AC_molecules)]
                #convert the labels and preds back to double type
                preds_grover['labels'] = preds_grover['labels'].astype(float)
                preds_grover['preds'] = preds_grover['preds'].astype(float)
                #drop NaN values
                preds_grover.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover['labels'], preds_grover['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                nonAC_perf_df = nonAC_perf_df.append(row_to_add, ignore_index=True)

                ##read grover_rdkit raw preds
                preds_grover_rdkit = pd.read_csv('../results/raw_predictions/grover/{task}/{mol_prop}/grover_base_rdkit/{split_type}/test_result_fold{fold}.csv'\
                                          .format(task=task_setting, mol_prop=mol_prop, split_type=split_type, fold=fold))
                #only keep the SMILES not in the AC set
                preds_grover_rdkit = preds_grover_rdkit[~preds_grover_rdkit['SMILES'].isin(AC_molecules)]
                #convert the labels and preds back to double type
                preds_grover_rdkit['labels'] = preds_grover_rdkit['labels'].astype(float)
                preds_grover_rdkit['preds'] = preds_grover_rdkit['preds'].astype(float)
                #drop NaN values
                preds_grover_rdkit.dropna(inplace=True)
                #calculate metric score
                score = metric_calc(preds_grover_rdkit['labels'], preds_grover_rdkit['preds'], metric_name)
                #assemble values to add
                values_to_add = {'metric_score': score, 'metric_name': metric_name, 'mol_prop': mol_prop, \
                                 'model_name': 'grover_base_rdkit', 'fold': fold, 'split_type': split_type, 'task': task_setting}
                #convert to a row series
                row_to_add = pd.Series(values_to_add)
                #append new row
                nonAC_perf_df = nonAC_perf_df.append(row_to_add, ignore_index=True)

-  combine the results for AC and non-AC molecules

In [29]:
#add the AC_label
AC_perf_df['AC_label'] = 'AC'
nonAC_perf_df['AC_label'] = 'non-AC'

#combine the AC and nonAC perf_df
combined_perf_df = pd.concat([AC_perf_df, nonAC_perf_df])

In [30]:
#save as a new grand_perf_df
combined_perf_df.to_csv('../results/processed_performance/AC_{folder}_grand_perf_df_{task}.csv'\
                        .format(folder=folder, task=task_setting), index=False)